In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as mlt
import seaborn as sp
from torch.autograd import Variable
from torch import autograd
from datetime import datetime
import matplotlib.pyplot as plt
from datetime import timedelta
import sys
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from scipy.stats import ks_2samp
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from types import SimpleNamespace
import json

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
%run ../Pre_process/Data_preprocess.ipynb
%run Model/Mogrifier_LSTM.ipynb
%run Model/Bi_LSTM.ipynb
%run Model/LSTM.ipynb
%run Model/SciNet.ipynb
%run train.ipynb
%run ../visualize.ipynb
%run test.ipynb
%run ../validation.ipynb
%run wrapper.ipynb
%run helper.ipynb

In [ ]:
with open("config/config_model.json") as json_data:
    data = json.load(json_data, object_hook=lambda d: SimpleNamespace(**d))
    parameters_bilstm = data.bilstm
    parameters_mogrifierlstm = data.mogrifierlstm
    parameters_scinet = data.scinet

In [ ]:
parameter_model = parameters_scinet
parameter_model.device = device

In [ ]:
wrapper = ModelTrain(parameter_model)
helper = ModelHelper(parameter_model)

In [ ]:
columns = ['Day of week', 'Energy', 'Week Day', 'Year', 'Month', 'Day of month', 'Hour', 'Sum']
#columns = ['Day of week', 'Energy', 'Week Day', 'Year', 'Month', 'Day of month', 'Hour', 'Sum', 'Seasonal', 'Trend', 'Residual']
# columns = ['Energy']

In [ ]:
data = pd.read_csv('../Data/Imputed/40_percent/Seq2Seq/caltech_last.csv')

In [ ]:
data['Hour'] = round(data['Hour'])

## Real Dataset Train Test

In [ ]:
data1 = pd.read_csv("../Data/Processed/palo_alto_data_with_zero.csv")
data2 = pd.read_csv("../Data/Processed/caltech_data_with_zero.csv")
data3 = pd.read_csv("../Data/Processed/jpl_data_with_zero.csv")
data4 = pd.read_csv("../Data/Processed/office_data_with_zero.csv")

data1_train = data1.loc[(data1['Start'] >= '2017-01-01 00:00:00') & (data1['Start'] <= '2019-09-30 23:00:00')].reset_index(drop=True)
data2_train = data2.loc[(data2['Start'] >= '2017-01-01 00:00:00') & (data2['Start'] <= '2019-09-30 23:00:00')].reset_index(drop=True)
data3_train = data3.loc[(data3['Start'] >= '2017-01-01 00:00:00') & (data3['Start'] <= '2019-09-30 23:00:00')].reset_index(drop=True)
data4_train = data4.loc[(data4['Start'] >= '2017-01-01 00:00:00') & (data4['Start'] <= '2019-09-30 23:00:00')].reset_index(drop=True)

df = pd.concat([data1_train, data2_train, data3_train, data4_train], axis=0).reset_index(drop=True)

df_real_train = df.groupby('Start').agg({
            'Day of week': 'first',
            'Energy': 'mean',
            'Week Day': 'first',
            'Year': 'first',
            'Month': 'first',
            'Day of month': 'first'
        }).reset_index()

df_real_train['Hour'] = pd.to_datetime(df_real_train['Start']).dt.hour
df_real_train['Sum'] = df_real_train.groupby(pd.to_datetime(df_real_train['Start']).dt.date)['Energy'].cumsum()

# df_real_train['Start'] = pd.to_datetime(df_real_train['Start'])
# df_real_train.set_index('Start', inplace=True)
# result = seasonal_decompose(df_real_train['Energy'], model='additive', extrapolate_trend='freq')
# df_real_train['Seasonal'] = result.seasonal
# df_real_train['Trend'] = result.trend
# df_real_train['Residual'] = result.resid
# df_real_train.reset_index(inplace=True)

df_real_train.drop(columns=['Start'], inplace=True)
#df_real_train.drop(columns=['Start', 'Year', 'Day of week', 'Week Day', 'Month', 'Day of month'], inplace=True)


In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
df_real_train = scaler.fit_transform(df_real_train)
df_real_train = pd.DataFrame(df_real_train, columns=columns)

In [ ]:
data_train, ground_truth_train = get_train_test_dataset_forecasting(df_real_train, parameter_model.lag_size, parameter_model.future_step)

In [ ]:
data1_test = data1.loc[(data1['Start'] >= '2019-10-01 00:00:00') & (data1['Start'] <= '2020-02-29 23:00:00')].reset_index(drop=True)
data2_test = data2.loc[(data2['Start'] >= '2019-10-01 00:00:00') & (data2['Start'] <= '2020-02-29 23:00:00')].reset_index(drop=True)
data3_test = data3.loc[(data3['Start'] >= '2019-10-01 00:00:00') & (data3['Start'] <= '2020-02-29 23:00:00')].reset_index(drop=True)
data4_test = data4.loc[(data4['Start'] >= '2019-10-01 00:00:00') & (data4['Start'] <= '2020-02-29 23:00:00')].reset_index(drop=True)

df = pd.concat([data1_test, data2_test, data3_test, data4_test], axis=0).reset_index(drop=True)

df_real_test = df.groupby('Start').agg({
            'Day of week': 'first',
            'Energy': 'mean',
            'Week Day': 'first',
            'Year': 'first',
            'Month': 'first',
            'Day of month': 'first'
        }).reset_index()

df_real_test['Hour'] = pd.to_datetime(df_real_test['Start']).dt.hour
df_real_test['Sum'] = df_real_test.groupby(pd.to_datetime(df_real_test['Start']).dt.date)['Energy'].cumsum()

# df_real_test['Start'] = pd.to_datetime(df_real_test['Start'])
# df_real_test.set_index('Start', inplace=True)
# result = seasonal_decompose(df_real_test['Energy'], model='additive', extrapolate_trend='freq')
# df_real_test['Seasonal'] = result.seasonal
# df_real_test['Trend'] = result.trend
# df_real_test['Residual'] = result.resid
# df_real_test.reset_index(inplace=True)

df_real_test.drop(columns=['Start'], inplace=True)
#df_real_test.drop(columns=['Start', 'Year', 'Day of week', 'Week Day', 'Month', 'Day of month'], inplace=True)

In [ ]:
scaler_test = MinMaxScaler(feature_range=(0,1))
df_real_test = scaler_test.fit_transform(df_real_test)
df_real_test = pd.DataFrame(df_real_test, columns=columns)

In [ ]:
data_test, ground_truth_test = get_train_test_dataset_forecasting(df_real_test, parameter_model.lag_size, parameter_model.future_step)

In [ ]:
step_per_epoch = len(data_train) // batch_size

# Converting to tensor
data_train = torch.from_numpy(data_train).float().to(device)
ground_truth_train = torch.from_numpy(ground_truth_train).float().to(device)

## Imputed Test

In [ ]:
model_type = "knn"

In [ ]:
data1 = pd.read_csv("../Data/Imputed/50_percent/" + model_type + "/palo_alto_final.csv")
data2 = pd.read_csv("../Data/Imputed/50_percent/" + model_type + "/caltech_final.csv")
data3 = pd.read_csv("../Data/Imputed/50_percent/" + model_type + "/jpl_final.csv")
data4 = pd.read_csv("../Data/Imputed/50_percent/" + model_type + "/office_final.csv")

df = pd.concat([data1, data2, data3, data4], axis=0).reset_index(drop=True)
df['Hour'] = round(df['Hour'])
df = df.rename(columns={'Day of month': 'Day_of_month'}) 

df['Start'] = pd.to_datetime(dict(year=round(df.Year), month=round(df.Month), day=round(df.Day_of_month), hour=round(df.Hour)))

df = df.loc[df['Start'] <= '2020-12-31 23:00:00'].reset_index(drop=True)

df = df.rename(columns={'Day_of_month': 'Day of month'}) 

df_real_test = df.groupby('Start').agg({
            'Day of week': 'first',
            'Energy': 'mean',
            'Week Day': 'first',
            'Year': 'first',
            'Month': 'first',
            'Day of month': 'first'
        }).reset_index()

df_real_test['Hour'] = pd.to_datetime(df_real_test['Start']).dt.hour
df_real_test['Sum'] = df_real_test.groupby(pd.to_datetime(df_real_test['Start']).dt.date)['Energy'].cumsum()
df_real_test.drop(columns=['Start'], inplace=True)

scaler_test = MinMaxScaler(feature_range=(0,1))
df_real_test = scaler_test.fit_transform(df_real_test)
df_real_test = pd.DataFrame(df_real_test, columns=columns)

data_test, ground_truth_test = get_train_test_dataset_forecasting(df_real_test, parameter_model.lag_size, parameter_model.future_step)

# Mogrifier LSTM

Trainning------------------------------------------------------------------------------------------------------------------------------

In [ ]:
model = Mogrifier_LSTM(parameter_model).to(device)
#model = Seq2SeqAttention(input_size, hidden_size, input_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = parameter_model.learning_rate, weight_decay = 0.005)
loss_function_mogrifier = nn.MSELoss()

In [ ]:
real_dataset_mogrifier, gen_dataset_mogrifier, errors_generator_mogrifier = wrapper.train_mogrifier_lstm(model, optimizer, loss_function_mogrifier, data_train, ground_truth_train, step_per_epoch, helper)

In [ ]:
gen_dataset_mogrifier.shape

In [ ]:
real_dataset_mogrifier.shape

In [ ]:
mlt.suptitle('Loss')
mlt.plot(errors_generator_mogrifier, label='d_loss')
mlt.legend()
#mlt.savefig('foo1.png')
mlt.show()

In [ ]:
gen_dataset_mogrifier.view(batch_size, -1).shape

In [ ]:
plot_full_dataset(real_dataset_mogrifier[:,0, :].detach().cpu().numpy(), gen_dataset_mogrifier[:, 0, :].detach().cpu().numpy(), 600, 52000, 'Results/test1.png', 1)

Testing-------------------------------------------------------------------------------------------------------------------------------

In [ ]:
step_per_epoch = len(data_test) // parameter_model.batch_size

In [ ]:
data_test = torch.from_numpy(data_test).float().to(device)
ground_truth_test = torch.from_numpy(ground_truth_test).float().to(device)

In [ ]:
wrapper_test = ModelTest(parameter_model)

In [ ]:
real_dataset_test_mogrifier, predicted_dataset_mogrifier, loss = wrapper_test.test_model(model, data_test, ground_truth_test, loss_function_mogrifier, step_per_epoch, helper)

In [ ]:
plot_full_dataset(real_dataset_test_mogrifier[:, 0, :].detach().cpu().numpy(), predicted_dataset_mogrifier[:, 0, :].detach().cpu().numpy(), 500, 10000, 'Results/test3.png', 1)

In [ ]:
mlt.suptitle('Loss')
#mlt.plot(errors_generator, label='train_loss')
mlt.plot(loss[:], label='test_loss')
mlt.legend()
#mlt.savefig('foo1.png')
mlt.show()

Evaluation----------------------------------------------------------------------------------------------------------------

In [ ]:
real_dataset_test_mogrifier = pd.DataFrame(real_dataset_test_mogrifier[:, 0, :].detach().cpu().numpy(), columns=columns)
real_dataset_test_mogrifier = scaler_test.inverse_transform(real_dataset_test_mogrifier)
real_dataset_test_mogrifier = torch.tensor(real_dataset_test_mogrifier)

In [ ]:
predicted_dataset_mogrifier = pd.DataFrame(predicted_dataset_mogrifier[:, 0, :].detach().cpu().numpy(), columns=columns)
predicted_dataset_mogrifier = scaler_test.inverse_transform(predicted_dataset_mogrifier)
predicted_dataset_mogrifier = torch.tensor(predicted_dataset_mogrifier)

In [ ]:
validation_matrix_forecasting(real_dataset_test_mogrifier[:, 1].detach().cpu().numpy(), predicted_dataset_mogrifier[:, 1].detach().cpu().numpy(), 0)

# Bi LSTM

In [ ]:
model = Bi_LSTM(parameter_model).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = parameter_model.learning_rate, weight_decay = 0.005)
loss_function_bilstm = nn.MSELoss()

In [ ]:
real_dataset_bilstm, gen_dataset_bilstm, errors_generator_bilstm = wrapper.train_bi_lstm(model, optimizer, loss_function_bilstm, data_train, ground_truth_train, step_per_epoch, helper)

In [ ]:
mlt.suptitle('Loss')
mlt.plot(errors_generator_bilstm, label='d_loss')
mlt.legend()
#mlt.savefig('foo1.png')
mlt.show()

In [ ]:
plot_full_dataset(real_dataset_bilstm[:, 0, :].detach().cpu().numpy(), gen_dataset_bilstm[:, 0, :].detach().cpu().numpy(), 00, 500, 'Results/test1.png', 1)

In [ ]:
data_test = data_test[: -(data_test.shape[0] % parameter_model.batch_size)]
ground_truth_test = ground_truth_test[: -(ground_truth_test.shape[0] % parameter_model.batch_size)]

In [ ]:
step_per_epoch = len(data_test) // parameter_model.batch_size

In [ ]:
data_test = torch.from_numpy(data_test).float().to(device)
ground_truth_test = torch.from_numpy(ground_truth_test).float().to(device)

In [ ]:
wrapper_test = ModelTest(parameter_model)

In [ ]:
real_data_bilstm, predicted_data_bilstm, loss_bilstm = wrapper_test.test_model(model, data_test, ground_truth_test, loss_function_bilstm, step_per_epoch, helper)

In [ ]:
plot_full_dataset(real_data_bilstm[:, 0, :].detach().cpu().numpy(), predicted_data_bilstm[:, 0, :].detach().cpu().numpy(), 000, 20000, 'Results/test3.png', 1)

In [ ]:
mlt.suptitle('Loss')
#mlt.plot(errors_generator, label='train_loss')
mlt.plot(loss_bilstm[:], label='test_loss')
mlt.legend()
#mlt.savefig('foo1.png')
mlt.show()

In [ ]:
real_data_bilstm = pd.DataFrame(real_data_bilstm[:, 0, :].detach().cpu().numpy(), columns=columns)
real_data_bilstm = scaler_test.inverse_transform(real_data_bilstm)
real_data_bilstm = torch.tensor(real_data_bilstm)

In [ ]:
predicted_data_bilstm = pd.DataFrame(predicted_data_bilstm[:, 0, :].detach().cpu().numpy(), columns=columns)
predicted_data_bilstm = scaler_test.inverse_transform(predicted_data_bilstm)
predicted_data_bilstm = torch.tensor(predicted_data_bilstm)

In [ ]:
validation_matrix_forecasting(real_data_bilstm[:, 1].detach().cpu().numpy(), predicted_data_bilstm[:, 1].detach().cpu().numpy(), 0)

# Vanilla LSTM

In [ ]:
hidden_size = 16
model = vanilla_LSTM(input_size, hidden_size, future_step).to(device)
#model = Seq2SeqAttention(input_size, hidden_size, input_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0.005)
loss_function_seq2seq = nn.MSELoss()

In [ ]:
real_dataset_vanilla_lstm, gen_dataset_vanilla_lstm, errors_generator_vanilla_lstm = train_vanilla_lstm(model, optimizer, loss_function_seq2seq, data_train, ground_truth_train, step_per_epoch)

In [ ]:
mlt.suptitle('Loss')
mlt.plot(errors_generator_vanilla_lstm, label='d_loss')
mlt.legend()
#mlt.savefig('foo1.png')
mlt.show()

In [ ]:
plot_full_dataset(real_dataset_vanilla_lstm[:].detach().cpu().numpy(), gen_dataset_vanilla_lstm[:].detach().cpu().numpy(), 00, 50000, 'Results/test1.png', 0)

In [ ]:
df = pd.read_csv("../Data/Imputed/Mean_imputation/perth.csv")
scaler = MinMaxScaler(feature_range=(0,1))
df = scaler.fit_transform(df)
df = pd.DataFrame(df, columns=['Day of week', 'Energy', 'Week Day', 'Year', 'Month', 'Day of month', 'Hour', 'Sum'])

In [ ]:
train_ratio = 0

In [ ]:
_, data_test, _, ground_truth_test = get_train_test_dataset_forecasting(df, train_ratio)

In [ ]:
data_test = data_test[: -(data_test.shape[0] % batch_size)]
ground_truth_test = ground_truth_test[: -(ground_truth_test.shape[0] % batch_size)]

In [ ]:
step_per_epoch = len(data_test) // batch_size

In [ ]:
data_test = torch.from_numpy(data_test).float().to(device)
ground_truth_test = torch.from_numpy(ground_truth_test).float().to(device)

In [ ]:
real_data_lstm, predicted_data_lstm, loss_lstm = test_model(model, data_test, ground_truth_test, loss_function_seq2seq, step_per_epoch)

In [ ]:
plot_full_dataset(real_data_lstm.detach().cpu().numpy(), predicted_data_lstm.detach().cpu().numpy(), 000, 5000, 'Results/test3.png', 0)

In [ ]:
mlt.suptitle('Loss')
#mlt.plot(errors_generator, label='train_loss')
mlt.plot(loss_lstm[:], label='test_loss')
mlt.legend()
#mlt.savefig('foo1.png')
mlt.show()

In [ ]:
validation_matrix_forecasting(real_data_lstm.view(-1).detach().cpu().numpy(), predicted_data_lstm.view(-1).detach().cpu().numpy(), 0)

In [ ]:
predicted_data_lstm.shape

In [ ]:
compare_predicted_dataset(real_data_lstm.detach().cpu().numpy(), predicted_dataset_mogrifier.detach().cpu().numpy(), predicted_data_bilstm.detach().cpu().numpy(), predicted_data_lstm.detach().cpu().numpy(), 000, 50, 'Results/compare.png', 0)

# Scinet

In [ ]:
model = SCINet(output_len = parameter_model.future_step, input_len= parameter_model.lag_size, input_dim = parameter_model.input_size, hid_size = hidden_size, num_stacks = parameter_model.stacks,
                num_levels = parameter_model.levels, concat_len = 0, groups = 1, kernel = 4, dropout = 0.2,
                 single_step_output_One = 0, positionalE =  False, modified = True).to(device)
#model = Seq2SeqAttention(input_size, hidden_size, input_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = parameter_model.learning_rate, weight_decay = 0.005)
loss_function_scinet = nn.L1Loss()

In [ ]:
real_dataset_scinet, gen_dataset_scinet, errors_generator_scinet = wrapper.train_scinet(model, loss_function_scinet, optimizer, data_train, ground_truth_train, step_per_epoch, helper)

In [ ]:
plot_full_dataset(real_dataset_scinet[:, 0, :].detach().cpu().numpy(), gen_dataset_scinet[:, 0, :].detach().cpu().numpy(), 300, 8000, 'Results/test1.png', 1)

test-----------------------------------------

In [ ]:
data_test = data_test[: -(data_test.shape[0] % parameter_model.batch_size)]
ground_truth_test = ground_truth_test[: -(ground_truth_test.shape[0] % parameter_model.batch_size)]

In [ ]:
step_per_epoch = len(data_test) // parameter_model.batch_size

In [ ]:
data_test = torch.from_numpy(data_test).float().to(device)
ground_truth_test = torch.from_numpy(ground_truth_test).float().to(device)

In [ ]:
wrapper_test = ModelTest(parameter_model)

In [ ]:
real_data_scinet, predicted_data_scinet, loss_scinet = wrapper_test.test_model(model, data_test, ground_truth_test, loss_function_scinet, step_per_epoch, helper)

In [ ]:
plot_full_dataset(real_data_scinet[:, 0, :].detach().cpu().numpy(), predicted_data_scinet[:, 0, :].detach().cpu().numpy(), 000, 50000, 'Results/test3.png', 1)

In [ ]:
mlt.suptitle('Loss')
#mlt.plot(errors_generator, label='train_loss')
mlt.plot(loss_scinet[:], label='test_loss')
mlt.legend()
#mlt.savefig('foo1.png')
mlt.show()

In [ ]:
real_data_scinet = pd.DataFrame(real_data_scinet[:, 0, :].detach().cpu().numpy(), columns=columns)
real_data_scinet = scaler_test.inverse_transform(real_data_scinet)
real_data_scinet = torch.tensor(real_data_scinet)

In [ ]:
predicted_data_scinet = pd.DataFrame(predicted_data_scinet[:, 0, :].detach().cpu().numpy(), columns=columns)
predicted_data_scinet = scaler_test.inverse_transform(predicted_data_scinet)
predicted_data_scinet = torch.tensor(predicted_data_scinet)

In [ ]:
validation_matrix_forecasting(real_data_scinet[:, 1].detach().cpu().numpy(), predicted_data_scinet[:, 1].detach().cpu().numpy(), 0)

In [ ]:
impute_folders = ['Seq2Seq', 'Mean_imputation', 'Knn']
datasets = ['acn', 'boulder', 'dundee', 'palo_alto', 'perth', 'sap']

In [ ]:
impute_folders = ['Processed']
datasets = ['acn_data_with_zero', 'boulder_data_with_zero', 'dundee_data_with_zero', 'palo_alto_data_with_zero', 'perth_data_with_zero', 'sap_data_with_zero']

In [ ]:
model.eval()

In [ ]:
for i in impute_folders:
    print(i + "---->")
    for j in datasets:
        print(j + ":")
        df = pd.read_csv("../Data/Imputed/" + i + "/" + j + ".csv")
        #df = df[: 8760]
        # df.drop(columns=['Start', 'Year', 'Day of week', 'Week Day', 'Month', 'Day of month'], inplace=True)
        df.drop(columns=['Day of week', 'Week Day', 'Year', 'Month', 'Day of month', 'Hour', 'Sum'], inplace=True)
        scaler = MinMaxScaler(feature_range=(0,1))
        df = scaler.fit_transform(df)
        df = pd.DataFrame(df, columns=columns)

        train_ratio = 0

        _, data_test, _, ground_truth_test = get_train_test_dataset_forecasting(df, train_ratio)

        # data_test = data_test[: -(data_test.shape[0] % batch_size)]
        # ground_truth_test = ground_truth_test[: -(ground_truth_test.shape[0] % batch_size)]

        data_test = torch.from_numpy(data_test).float().to(device)
        ground_truth_test = torch.from_numpy(ground_truth_test).float().to(device)

        with torch.no_grad():
            if stacks == 1:
                outputs = model(data_test)
            elif stacks == 2:
                outputs, _ = model(data_test)

        ground_truth_test = pd.DataFrame(ground_truth_test[:, -1, :].detach().cpu().numpy(), columns=columns)
        ground_truth_test = scaler.inverse_transform(ground_truth_test)
        ground_truth_test = torch.tensor(ground_truth_test)

        outputs = pd.DataFrame(outputs[:, -1, :].detach().cpu().numpy(), columns=columns)
        outputs = scaler.inverse_transform(outputs)
        outputs = torch.tensor(outputs)

        validation_matrix_forecasting(ground_truth_test.detach().cpu().numpy(), outputs.detach().cpu().numpy(), 1)


In [ ]:
df = pd.read_csv("../Data/Processed/acn_data_with_zero.csv")
df = df[: 8760]
df.drop(columns=['Start', 'Year', 'Day of week', 'Week Day', 'Month', 'Day of month'], inplace=True)
scaler = MinMaxScaler(feature_range=(0,1))
df = scaler.fit_transform(df)
df = pd.DataFrame(df, columns=columns)

In [ ]:
train_ratio = 0

In [ ]:

_, data_test, _, ground_truth_test = get_train_test_dataset_forecasting(df, train_ratio)

In [ ]:
data_test = data_test[: -(data_test.shape[0] % batch_size)]
ground_truth_test = ground_truth_test[: -(ground_truth_test.shape[0] % batch_size)]

In [ ]:
data_test = torch.from_numpy(data_test).float().to(device)
ground_truth_test = torch.from_numpy(ground_truth_test).float().to(device)

In [ ]:
model.eval()
with torch.no_grad():
    if stacks == 1:
        outputs = model(data_test)
    elif stacks == 2:
        outputs, _ = model(data_test)

In [ ]:
outputs.shape

In [ ]:
ground_truth_test = pd.DataFrame(ground_truth_test[:, -1, :].detach().cpu().numpy(), columns=['Energy'])
ground_truth_test = scaler.inverse_transform(ground_truth_test)
ground_truth_test = torch.tensor(ground_truth_test)

outputs = pd.DataFrame(outputs[:, -1, :].detach().cpu().numpy(), columns=['Energy'])
outputs = scaler.inverse_transform(outputs)
outputs = torch.tensor(outputs)

In [ ]:
ground_truth_test.shape

In [ ]:
validation_matrix_forecasting(ground_truth_test.detach().cpu().numpy(), outputs.detach().cpu().numpy(), 1)

In [ ]:
ground_truth_test.shape

In [ ]:
plot_full_dataset(ground_truth_test.detach().cpu().numpy(), outputs.detach().cpu().numpy(), 1000, 1500, 'Results/test1.png', 0)

In [ ]:
results = {
    '1 Hour': {'MoG-LSTM': 0.55, 'BiLSTM': 0.57, 'LSTM': 0.57},
    '4 Hour': {'MoG-LSTM': 0.29, 'BiLSTM': 0.23, 'LSTM': 0.27},
    '8 Hour': {'MoG-LSTM': 0.25, 'BiLSTM': 0.12, 'LSTM': 0.14}
}
plot_r2_score(results, "Results/compare_hourly.png")

In [ ]:
random_row_indices = np.random.choice(100, size=40, replace=False)

In [ ]:
random_row_indices